In [1]:
import numpy as np
import torch
import deepchem as dc
from torch_geometric.data import Data, DataLoader
import pandas as pd

#check GPU
print('CUDA available: {}'.format(torch.cuda.is_available()))
print('Current GPU: {}'.format(torch.cuda.get_device_name(torch.cuda.current_device())))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

RDKit WARNING: [21:18:10] Enabling RDKit 2019.09.3 jupyter extensions


CUDA available: True
Current GPU: GeForce RTX 2080 SUPER


In [2]:
#cis_model = torch.load('trained_models/AFP_tl_cisplatin_0.9082')
cis_model = torch.load('trained_models/AFP_tl_cisplatin_test')
#amg_model = torch.load('trained_models/AFP_tl_amg_0.8836')
amg_model = torch.load('trained_models/AFP_tl_amg_test')

In [3]:
from featurizer import MolGraphConvFeaturizer

featurizer = MolGraphConvFeaturizer(
    use_edges=True, use_chirality=True, use_partial_charge=True
)

In [4]:
from rdkit import Chem
from utils import validate_mols

def get_valid_smiles(data):
    smiles = list(data['smiles'])
    mols = [Chem.MolFromSmiles(s) for s in smiles]
    valid_mols, valid_index = validate_mols(mols)
    valid_smiles = [Chem.MolToSmiles(mol) for mol in valid_mols]
    return valid_smiles, valid_index

def get_valid_mols(data):
    smiles = list(data['smiles'])
    mols = [Chem.MolFromSmiles(s) for s in smiles]
    valid_mols, valid_index = validate_mols(mols)
    return valid_mols, valid_index

def get_valid_features(molecules, featurizer):
    valid_features = []
    valid_index = []
    for i, mol in enumerate(molecules):
        try:
            valid_features.append(featurizer._featurize(mol))
            valid_index.append(i)
        except Exception as e:
            print("Failed to featurize datapoint %d, %s.", i, Chem.MolToSmiles(mol))
            print("Exception message: {}".format(e))

    return np.asarray(valid_features), np.array(valid_index)

def featurize_data(data, featurizer):
    valid_smiles, valid_index = get_valid_smiles(data)
    valid_mols, valid_index = get_valid_mols(data)
    valid_data = data.iloc[valid_index]
    valid_features, valid_index = get_valid_features(valid_mols, featurizer)
    valid_data = valid_data.iloc[valid_index]
    valid_data['smiles'] = valid_smiles
    return valid_features, valid_data

def predict(model, loader):
    model.eval()

    outs = []
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        outs.append(torch.sigmoid(out).detach().cpu().numpy())
    
    pred = np.concatenate(outs, axis=0)
    return pred

def check_training_data(train_dataset, res_data):
    for i in range(len(res_data)):
        curr_smiles = res_data.iloc[i]['smiles']
        for j, tr_smiles in enumerate(train_dataset.ids):
            if tr_smiles == curr_smiles:
                if train_dataset.y[j][0] == 1.:
                    res_data.iloc[i, 2] = -1.
                else:
                    res_data.iloc[i, 2] = -2.
    return res_data

In [5]:
amg_data = pd.read_csv('data/tl/amg_data-210723.csv')
valid_amg_features, valid_amg_data = featurize_data(
    amg_data, featurizer)

amg_loader = DataLoader([x.to_pyg_graph() for x in valid_amg_features], 
           batch_size=16, shuffle=False)
amg_pred = predict(amg_model, amg_loader)
res_amg_data = valid_amg_data.copy()
res_amg_data.insert(2, 'p', amg_pred)
res_amg_data_sorted = res_amg_data.sort_values(by=['p'], ascending=False)
res_amg_data_sorted

,Name,Molecular Formula,p,smiles,Molecular weight,Reference
9,coenzyme Q10,C59H90O4,0.911192,COC1=C(OC)C(=O)C(CC=C(C)CCC=C(C)CCC=C(C)CCC=C(...,863.30,Mechanism and Prevention of Ototoxicity Induce...
14,Memantine,C12H21N,0.884105,CC12CC3CC(C)(C1)CC(N)(C3)C2,179.30,Mechanism and Prevention of Ototoxicity Induce...
30,tacrolimus,C44H69NO12,0.880260,C=CCC1C=C(C)CC(C)CC(OC)C2OC(O)(C(=O)C(=O)N3CCC...,804.00,Efficacy of three drugs for protecting against...
10,alpha-tocopherol,C29H50O2,0.879468,Cc1c(C)c2c(c(C)c1O)CCC(C)(CCCC(C)CCCC(C)CCCC(C...,430.70,Mechanism and Prevention of Ototoxicity Induce...
67,hernandezine,C39H44N2O7,0.879159,COc1ccc2cc1Oc1ccc(cc1)CC1c3cc(c(OC)cc3CCN1C)Oc...,652.80,Natural Bizbenzoquinoline Derivatives Protect ...
...,...,...,...,...,...,...
7,deferoxamine,C25H48N6O8,0.428576,CC(=O)N(O)CCCCCNC(=O)CCC(=O)N(O)CCCCCNC(=O)CCC...,560.70,Mechanism and Prevention of Ototoxicity Induce...
28,adjudin,C15H12Cl2N4O,0.393844,NNC(=O)c1nn(Cc2ccc(Cl)cc2Cl)c2ccccc12,335.20,Adjudin protects rodent cochlear hair cells ag...
63,2-APB,C14H16BNO,0.342282,NCCOB(c1ccccc1)c1ccccc1,225.10,Identification of ion-channel modulators that ...
34,Sodium selenite,Na2O3Se,0.158199,O=[Se]([O-])[O-].[Na+].[Na+],172.95,Sodium Selenite Acts as an Otoprotectant again...


In [6]:
loaded, datasets, transformers = dc.utils.load_dataset_from_disk('data/combined/amg')
train_dataset, valid_dataset, test_dataset = datasets

In [7]:
check_training_data(train_dataset, res_amg_data_sorted)

,Name,Molecular Formula,p,smiles,Molecular weight,Reference
9,coenzyme Q10,C59H90O4,-1.000000,COC1=C(OC)C(=O)C(CC=C(C)CCC=C(C)CCC=C(C)CCC=C(...,863.30,Mechanism and Prevention of Ototoxicity Induce...
14,Memantine,C12H21N,-1.000000,CC12CC3CC(C)(C1)CC(N)(C3)C2,179.30,Mechanism and Prevention of Ototoxicity Induce...
30,tacrolimus,C44H69NO12,-1.000000,C=CCC1C=C(C)CC(C)CC(OC)C2OC(O)(C(=O)C(=O)N3CCC...,804.00,Efficacy of three drugs for protecting against...
10,alpha-tocopherol,C29H50O2,0.879468,Cc1c(C)c2c(c(C)c1O)CCC(C)(CCCC(C)CCCC(C)CCCC(C...,430.70,Mechanism and Prevention of Ototoxicity Induce...
67,hernandezine,C39H44N2O7,-1.000000,COc1ccc2cc1Oc1ccc(cc1)CC1c3cc(c(OC)cc3CCN1C)Oc...,652.80,Natural Bizbenzoquinoline Derivatives Protect ...
...,...,...,...,...,...,...
7,deferoxamine,C25H48N6O8,0.428576,CC(=O)N(O)CCCCCNC(=O)CCC(=O)N(O)CCCCCNC(=O)CCC...,560.70,Mechanism and Prevention of Ototoxicity Induce...
28,adjudin,C15H12Cl2N4O,-1.000000,NNC(=O)c1nn(Cc2ccc(Cl)cc2Cl)c2ccccc12,335.20,Adjudin protects rodent cochlear hair cells ag...
63,2-APB,C14H16BNO,0.342282,NCCOB(c1ccccc1)c1ccccc1,225.10,Identification of ion-channel modulators that ...
34,Sodium selenite,Na2O3Se,0.158199,O=[Se]([O-])[O-].[Na+].[Na+],172.95,Sodium Selenite Acts as an Otoprotectant again...


In [8]:
res_amg_data_sorted.to_csv('data/tl/amg_data2.csv', index=False)

In [9]:
fda_test_data = pd.read_csv('data/tl/FDA_tested_data-210723.csv')
valid_fda_test_features, valid_fda_test_data = featurize_data(
    fda_test_data, featurizer)

fda_test_loader = DataLoader([x.to_pyg_graph() for x in valid_fda_test_features], 
           batch_size=16, shuffle=False)
fda_test_pred = predict(amg_model, fda_test_loader)
res_fda_test_data = valid_fda_test_data.copy()
res_fda_test_data.insert(2, 'p', fda_test_pred)
res_fda_test_data_sorted = res_fda_test_data.sort_values(by=['p'], ascending=False)
res_fda_test_data_sorted

,Name,smiles,p,amg,cisplatin,Reference
5,phenoxybenzamine,CC(COc1ccccc1)N(CCCl)Cc1ccccc1,0.842589,1,0,Screen of FDA-approved drug library reveals co...
6,loperamide,CN(C)C(=O)C(CCN1CCC(O)(c2ccc(Cl)cc2)CC1)(c1ccc...,0.833367,1,0,Screen of FDA-approved drug library reveals co...
3,fluoxetine,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1,0.766812,1,0,Screen of FDA-approved drug library reveals co...
4,fluspirilene,O=C1NCN(c2ccccc2)C12CCN(CCCC(c1ccc(F)cc1)c1ccc...,0.751552,1,0,Screen of FDA-approved drug library reveals co...
7,benzamil,NC(=NCc1ccccc1)NC(=O)c1nc(Cl)c(N)nc1N,0.631011,1,1,Screen of FDA-approved drug library reveals co...
1,tamoxifen,CC/C(=C(\c1ccccc1)c1ccc(OCCN(C)C)cc1)c1ccccc1,0.571418,1,0,Screen of FDA-approved drug library reveals co...
2,paroxetine,Fc1ccc([C@@H]2CCNC[C@H]2COc2ccc3c(c2)OCO3)cc1,0.235836,1,1,Screen of FDA-approved drug library reveals co...
0,raloxifene,O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2...,0.072860,1,0,Screen of FDA-approved drug library reveals co...


In [10]:
check_training_data(train_dataset, res_fda_test_data_sorted)

,Name,smiles,p,amg,cisplatin,Reference
5,phenoxybenzamine,CC(COc1ccccc1)N(CCCl)Cc1ccccc1,-2.000000,1,0,Screen of FDA-approved drug library reveals co...
6,loperamide,CN(C)C(=O)C(CCN1CCC(O)(c2ccc(Cl)cc2)CC1)(c1ccc...,0.833367,1,0,Screen of FDA-approved drug library reveals co...
3,fluoxetine,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1,0.766812,1,0,Screen of FDA-approved drug library reveals co...
4,fluspirilene,O=C1NCN(c2ccccc2)C12CCN(CCCC(c1ccc(F)cc1)c1ccc...,0.751552,1,0,Screen of FDA-approved drug library reveals co...
7,benzamil,NC(=NCc1ccccc1)NC(=O)c1nc(Cl)c(N)nc1N,0.631011,1,1,Screen of FDA-approved drug library reveals co...
1,tamoxifen,CC/C(=C(\c1ccccc1)c1ccc(OCCN(C)C)cc1)c1ccccc1,0.571418,1,0,Screen of FDA-approved drug library reveals co...
2,paroxetine,Fc1ccc([C@@H]2CCNC[C@H]2COc2ccc3c(c2)OCO3)cc1,0.235836,1,1,Screen of FDA-approved drug library reveals co...
0,raloxifene,O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2...,0.072860,1,0,Screen of FDA-approved drug library reveals co...


In [11]:
pos_data = pd.read_csv('data/tl/experiment_check_positive_data.csv')
valid_pos_features, valid_pos_data = featurize_data(
    pos_data, featurizer)
pos_loader = DataLoader([x.to_pyg_graph() for x in valid_pos_features], 
           batch_size=16, shuffle=False)

pos_pred = predict(amg_model, pos_loader)

res_pos_data = valid_pos_data.copy()
res_pos_data.insert(2, 'p', pos_pred)
res_pos_data_sorted = res_pos_data.sort_values(by=['p'], ascending=False)
res_pos_data_sorted

,Name,smiles,p
3,Dexpanthenol,CC(C)(CO)C(O)C(=O)NCCCO,0.837283
4,Midodrine Hydrochloride,COc1ccc(OC)c(C(O)CNC(=O)CN)c1.Cl,0.831568
0,Ammonium Lactate solution,CC(O)C(=O)[O-].[NH4+],0.801609
1,L-Glutamine,NC(=O)CCC(N)C(=O)O,0.727602
5,Calcium Citrate Tetrahydrate,O.O.O.O.O=C([O-])CC(O)(CC(=O)[O-])C(=O)[O-].O=...,0.673695
2,Malic Acid,O=C(O)CC(O)C(=O)O,0.637007
6,Strontium Ranelate,N#Cc1c(N(CC(=O)[O-])CC(=O)[O-])sc(C(=O)[O-])c1...,0.111484


In [12]:
check_training_data(train_dataset, res_pos_data_sorted)

,Name,smiles,p
3,Dexpanthenol,CC(C)(CO)C(O)C(=O)NCCCO,0.837283
4,Midodrine Hydrochloride,COc1ccc(OC)c(C(O)CNC(=O)CN)c1.Cl,0.831568
0,Ammonium Lactate solution,CC(O)C(=O)[O-].[NH4+],0.801609
1,L-Glutamine,NC(=O)CCC(N)C(=O)O,0.727602
5,Calcium Citrate Tetrahydrate,O.O.O.O.O=C([O-])CC(O)(CC(=O)[O-])C(=O)[O-].O=...,0.673695
2,Malic Acid,O=C(O)CC(O)C(=O)O,0.637007
6,Strontium Ranelate,N#Cc1c(N(CC(=O)[O-])CC(=O)[O-])sc(C(=O)[O-])c1...,0.111484


In [13]:
neg_data = pd.read_csv('data/tl/experiment_negative_data.csv')
valid_neg_features, valid_neg_data = featurize_data(
    neg_data, featurizer)

neg_loader = DataLoader([x.to_pyg_graph() for x in valid_neg_features], 
           batch_size=16, shuffle=False)

neg_pred = predict(amg_model, neg_loader)

res_neg_data = valid_neg_data.copy()
res_neg_data.insert(2, 'p', neg_pred)
res_neg_data_sorted = res_neg_data.sort_values(by=['p'], ascending=False)
res_neg_data_sorted

,Name,smiles,p
8,O-Acetyl-L-Carnitine Hydrochloride,CC(=O)OC(CC(=O)O)C[N+](C)(C)C.[Cl-],0.833024
15,D-Minus-Penicillamine free base,CC(C)(S)C(N)C(=O)O,0.828038
19,Lacosamide,COCC(NC(C)=O)C(=O)NCc1ccccc1,0.824949
5,DL-Lactic Acid,CC(O)C(=O)O,0.787289
17,Selenomethionine,C[Se]CCC(N)C(=O)O,0.779856
3,Gamolenic Acid,CCCCCC=CCC=CCC=CCCCCC(=O)O,0.774660
6,Dihydroxyacetone,O=C(CO)CO,0.767901
2,Acamprosate,CC(=O)NCCCS(=O)(=O)O,0.765112
14,6-Aminocaproic Acid,NCCCCCC(=O)O,0.756546
11,Vigabatrin,C=CC(N)CCC(=O)O,0.744309


In [14]:
check_training_data(train_dataset, res_neg_data_sorted)

,Name,smiles,p
8,O-Acetyl-L-Carnitine Hydrochloride,CC(=O)OC(CC(=O)O)C[N+](C)(C)C.[Cl-],-2.000000
15,D-Minus-Penicillamine free base,CC(C)(S)C(N)C(=O)O,-2.000000
19,Lacosamide,COCC(NC(C)=O)C(=O)NCc1ccccc1,-2.000000
5,DL-Lactic Acid,CC(O)C(=O)O,-2.000000
17,Selenomethionine,C[Se]CCC(N)C(=O)O,-2.000000
3,Gamolenic Acid,CCCCCC=CCC=CCC=CCCCCC(=O)O,-2.000000
6,Dihydroxyacetone,O=C(CO)CO,-2.000000
2,Acamprosate,CC(=O)NCCCS(=O)(=O)O,0.765112
14,6-Aminocaproic Acid,NCCCCCC(=O)O,-2.000000
11,Vigabatrin,C=CC(N)CCC(=O)O,-2.000000


In [15]:
cis_data = pd.read_csv('data/tl/cisplatin_data-210723.csv')
valid_cis_features, valid_cis_data = featurize_data(
    cis_data, featurizer)

cis_loader = DataLoader([x.to_pyg_graph() for x in valid_cis_features], 
           batch_size=16, shuffle=False)
cis_pred = predict(cis_model, cis_loader)
res_cis_data = valid_cis_data.copy()
res_cis_data.insert(2, 'p', cis_pred)
res_cis_data_sorted = res_cis_data.sort_values(by=['p'], ascending=False)
res_cis_data_sorted

,Name,Molecular Formula,p,smiles,Molecular weight,Reference
20,Forskolin,C22H34O7,0.914218,C=CC1(C)CC(=O)C2(O)C(C)(O1)C(OC(C)=O)C(O)C1C(C...,410.50,Current Strategies to Combat Cisplatin-Induced...
58,Ganaxolone,C22H36O2,0.910921,CC(=O)C1CCC2C3CCC4CC(C)(O)CCC4(C)C3CCC12C,332.50,The identification of dual protective agents a...
47,ursolic acid,C30H48O3,0.906217,CC1CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CCC(O)C(C)(C...,456.70,Ursolic acid protects against cisplatin-induce...
38,Tauroursodeoxycholic acid (TUDCA),C26H45NO6S,0.905124,CC(CCC(=O)NCCS(=O)(=O)O)C1CCC2C3C(O)CC4CC(O)CC...,499.70,Current Strategies to Combat Cisplatin-Induced...
65,Ouabain,C29H44O12,0.903875,CC1OC(OC2CC(O)C3(CO)C4C(O)CC5(C)C(C6=CC(=O)OC6...,584.70,The identification of dual protective agents a...
...,...,...,...,...,...,...
31,AT7519,C16H17Cl2N5O2,0.340864,O=C(NC1CCNCC1)c1[nH]ncc1NC(=O)c1c(Cl)cccc1Cl,382.20,Current Strategies to Combat Cisplatin-Induced...
79,Dabrafenib,C23H20F3N5O2S2,0.338517,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,519.60,BRAF inhibition protects against hearing loss ...
42,Quercetin,C15H10O7,0.140098,O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,302.23,NaN
56,Apigenin,C15H10O5,0.118917,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,270.24,The identification of dual protective agents a...


In [17]:
check_training_data(train_dataset, res_cis_data_sorted)

,Name,Molecular Formula,p,smiles,Molecular weight,Reference
20,Forskolin,C22H34O7,0.914218,C=CC1(C)CC(=O)C2(O)C(C)(O1)C(OC(C)=O)C(O)C1C(C...,410.50,Current Strategies to Combat Cisplatin-Induced...
58,Ganaxolone,C22H36O2,0.910921,CC(=O)C1CCC2C3CCC4CC(C)(O)CCC4(C)C3CCC12C,332.50,The identification of dual protective agents a...
47,ursolic acid,C30H48O3,0.906217,CC1CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CCC(O)C(C)(C...,456.70,Ursolic acid protects against cisplatin-induce...
38,Tauroursodeoxycholic acid (TUDCA),C26H45NO6S,0.905124,CC(CCC(=O)NCCS(=O)(=O)O)C1CCC2C3C(O)CC4CC(O)CC...,499.70,Current Strategies to Combat Cisplatin-Induced...
65,Ouabain,C29H44O12,0.903875,CC1OC(OC2CC(O)C3(CO)C4C(O)CC5(C)C(C6=CC(=O)OC6...,584.70,The identification of dual protective agents a...
...,...,...,...,...,...,...
31,AT7519,C16H17Cl2N5O2,0.340864,O=C(NC1CCNCC1)c1[nH]ncc1NC(=O)c1c(Cl)cccc1Cl,382.20,Current Strategies to Combat Cisplatin-Induced...
79,Dabrafenib,C23H20F3N5O2S2,0.338517,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,519.60,BRAF inhibition protects against hearing loss ...
42,Quercetin,C15H10O7,-2.000000,O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,302.23,NaN
56,Apigenin,C15H10O5,-2.000000,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,270.24,The identification of dual protective agents a...


In [18]:
res_cis_data_sorted.to_csv('data/tl/cisplatin_data2.csv', index=False)

In [19]:
pos_pred = predict(cis_model, pos_loader)

res_pos_data = valid_pos_data.copy()
res_pos_data.insert(2, 'p', pos_pred)
res_pos_data_sorted = res_pos_data.sort_values(by=['p'], ascending=False)
res_pos_data_sorted

,Name,smiles,p
0,Ammonium Lactate solution,CC(O)C(=O)[O-].[NH4+],0.885634
3,Dexpanthenol,CC(C)(CO)C(O)C(=O)NCCCO,0.882398
5,Calcium Citrate Tetrahydrate,O.O.O.O.O=C([O-])CC(O)(CC(=O)[O-])C(=O)[O-].O=...,0.814756
2,Malic Acid,O=C(O)CC(O)C(=O)O,0.810004
1,L-Glutamine,NC(=O)CCC(N)C(=O)O,0.785046
4,Midodrine Hydrochloride,COc1ccc(OC)c(C(O)CNC(=O)CN)c1.Cl,0.704289
6,Strontium Ranelate,N#Cc1c(N(CC(=O)[O-])CC(=O)[O-])sc(C(=O)[O-])c1...,0.326831


In [20]:
neg_pred = predict(cis_model, neg_loader)

res_neg_data = valid_neg_data.copy()
res_neg_data.insert(2, 'p', neg_pred)
res_neg_data_sorted = res_neg_data.sort_values(by=['p'], ascending=False)
res_neg_data_sorted

,Name,smiles,p
15,D-Minus-Penicillamine free base,CC(C)(S)C(N)C(=O)O,0.900933
5,DL-Lactic Acid,CC(O)C(=O)O,0.890544
4,Potassium L-Lactate solution,CC(O)C(=O)[O-].[K+],0.856919
8,O-Acetyl-L-Carnitine Hydrochloride,CC(=O)OC(CC(=O)O)C[N+](C)(C)C.[Cl-],0.848716
17,Selenomethionine,C[Se]CCC(N)C(=O)O,0.833615
18,Etidronic Acid,CC(O)(P(=O)(O)O)P(=O)(O)O,0.814614
14,6-Aminocaproic Acid,NCCCCCC(=O)O,0.808168
13,Sodium L-Ascorbate,O=C1OC(C(O)CO)C([O-])=C1O.[Na+],0.786613
10,S-Carboxymethyl-L-Cysteine,NC(CSCC(=O)O)C(=O)O,0.775543
7,Hydroxyurea,NC(=O)NO,0.774460


In [21]:
fda_test_pred = predict(cis_model, fda_test_loader)
res_fda_test_data = valid_fda_test_data.copy()
res_fda_test_data.insert(2, 'p', fda_test_pred)
res_fda_test_data_sorted = res_fda_test_data.sort_values(by=['p'], ascending=False)
res_fda_test_data_sorted

,Name,smiles,p,amg,cisplatin,Reference
4,fluspirilene,O=C1NCN(c2ccccc2)C12CCN(CCCC(c1ccc(F)cc1)c1ccc...,0.841847,1,0,Screen of FDA-approved drug library reveals co...
5,phenoxybenzamine,CC(COc1ccccc1)N(CCCl)Cc1ccccc1,0.712931,1,0,Screen of FDA-approved drug library reveals co...
3,fluoxetine,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1,0.696641,1,0,Screen of FDA-approved drug library reveals co...
6,loperamide,CN(C)C(=O)C(CCN1CCC(O)(c2ccc(Cl)cc2)CC1)(c1ccc...,0.508074,1,0,Screen of FDA-approved drug library reveals co...
7,benzamil,NC(=NCc1ccccc1)NC(=O)c1nc(Cl)c(N)nc1N,0.393596,1,1,Screen of FDA-approved drug library reveals co...
1,tamoxifen,CC/C(=C(\c1ccccc1)c1ccc(OCCN(C)C)cc1)c1ccccc1,0.151098,1,0,Screen of FDA-approved drug library reveals co...
0,raloxifene,O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2...,0.093796,1,0,Screen of FDA-approved drug library reveals co...
2,paroxetine,Fc1ccc([C@@H]2CCNC[C@H]2COc2ccc3c(c2)OCO3)cc1,0.092595,1,1,Screen of FDA-approved drug library reveals co...


In [ ]:
#res_pos_data_sorted.to_csv('results/cisplatin_fda_approved_sorted-210720.csv', 
#                                    index=False)